In [1]:
from pathlib import Path
import pandas as pd
from gensim import models

In [2]:
st_file_en = "../tmp/Psycho-Paper/synonym_judgement/cueing study stimuli for distribution.csv"

df_full = pd.read_csv(st_file_en)
df = df_full[['Probe', 'Target', 'Foil1', 'Foil2']]
df = df[~df.isna().any(axis=1)]
df

,Probe,Target,Foil1,Foil2
0,abstain,refrain,covet,specify
1,accordance,agreement,substitute,analogy
2,advantage,benefit,tendency,condition
3,alias,pseudonym,aspect,reprisal
4,alternative,substitute,ambition,discretion
...,...,...,...,...
195,villain,crook,herring,aluminium
196,violin,viola,shed,rabbit
197,waist,torso,goddess,chorus
198,window,door,eye,plant


In [3]:
google_w2v = models.KeyedVectors.load_word2vec_format('../data/vectors/GoogleNews-vectors-negative300.bin', binary=True)

In [4]:
def closest_match(terms, vectors):
    """
    Returns the index of the term closest to the first term in a list of words.
    
    Note that index 0 is taken as the probe and all words with index > 0 are tested.
    """
    
    terms = terms.to_list()
    # print(terms)
    try:
        distances = vectors.distances(terms[0], terms[1:])
        # print(distances)
        min_dist = distances.argmin() + 1
        return min_dist
    except KeyError:
        return -1

    
def synonym_judgement_accuracy(wordvectors, tests=df, target_idx=1):
    pred = tests.apply(lambda x: closest_match(x, wordvectors), axis=1)
    pred = pred[pred > 0]
    correct = (pred == target_idx).sum()
    acc = correct / len(pred)
    print('Accuracy:', round(acc, 3))
    print('Tests with unknown words:', len(df) - len(pred))
    
synonym_judgement_accuracy(google_w2v)

Accuracy: 0.939
Tests with unknown words: 4


In [6]:
# convert csv file to w2v format
file = 'bnc_lsi_gensim_term_vectors.csv'
dir_path = Path('../data/out/SemD/bnc_cs1000_minsz50_lc_filtered')
file_path = dir_path / file
w2v_file_path = file_path.with_suffix('.w2v')

In [ ]:
lsi_wv = pd.read_csv(file_path, index_col=0)

with open(w2v_file_path, 'w') as fp:
    fp.write(f'{lsi_wv.shape[0]} {lsi_wv.shape[1]}\n')
    lsi_wv.to_csv(fp, sep=' ', header=None)

lsi_wv

In [7]:
bnc_lsi = models.KeyedVectors.load_word2vec_format(w2v_file_path)
bnc_lsi

In [8]:
synonym_judgement_accuracy(bnc_lsi)

Accuracy: 0.844
Tests with unknown words: 27
